In [ ]:
"""
Single-track vehicle dynamics with coupled-slip front and rear axles

x := [ux, uy, r, s, e, dpsi]   (n = 6)
u := [delta, kappa_f, kappa_r] (m = 3)

"""

G = 9.81 # m/s^2, gravitational accel

import matplotlib.pyplot as plt
import numpy as np

Basic Example

In [ ]:
# %% Initialization
# Vehicle parameters
# (all SI units: kg, m, rad, s, N, and combinations thereof).
params = {"m" : 1700,       # mass
          "Izz" : 2300,     # moment of inertia
          "a" : 1.2,        # distance from CoM to front axle
          "b" : 1.4,        # distance from CoM to rear axle
          "l" : 1.2 + 1.4,  # wheelbase
          "t" : 1.5,        # track width 
          "h_com" : 1.5,    # CoM height
          "Cx_f" : 275000,  # cornering stiffness front axle
          "Cy_f" : 100000,  # cornering stiffness front axle
          "Cx_r" : 300000,  # cornering stiffness rear axle
          "Cy_r" : 150000,  # cornering stiffness rear axle
          "mu_f" : 0.8,     # friction coefficient front axle
          "mu_r" : 0.9}     # friction coefficient rear axle

In [ ]:
def compute_coupled_slip_forces(kappa, alpha, Fz, Cx, Cy, mu):
    """
    Computes the tire forces for a given slip angle and slip ratio

    Inputs:
        kappa (slip ratio)
        alpha (slip angle)
        Fz (normal force)
        Cx (longitudinal stiffness)
        Cy (lateral stiffness)
        mu (friction coefficient)
    
    Returns:
        Fx (longitudinal tire force)
        Fy (lateral tire force)
    """
    sigma = np.sqrt(Cx**2*(kappa/(1+kappa))**2 + Cy**2*(np.tan(alpha)/(1+kappa))**2)
    
    if(sigma <= 3*mu*Fz):
        # Not fully saturated
        F = sigma - sigma**2/(3*mu*Fz) + sigma**3/(27*(mu*Fz)**2)
        
    else:
        # Fully saturated
        F = mu*Fz
    
    # Partition total contact patch force into longitudinal and lateral components
    if(sigma == 0):
        # No tire force
        Fx = 0
        Fy = 0
    
    else:
        Fx = F*(Cx*kappa/(sigma*(kappa+1)))
        Fy = -F*(Cy*np.tan(alpha)/(sigma*(kappa+1)))
    
    return Fx, Fy

In [ ]:
# %% Dynamics equations
def compute_x_dot(x, u, k_psi, params):
    """
    Computes the state derivative x_dot
    
    Inputs:
        x (state)
        u (control)
        k_psi (road curvature; is just zero for a straight lane)
        params (vehicle parameters dictionary)
        
    Returns:
        x_dot (state derivative)
        
    """
    
    # Unpack states
    ux      = x[0]  # longitudinal velocity
    uy      = x[1]  # lateral velocity
    r       = x[2]  # yaw rate
    s       = x[3]  # path-progress coordinate
    e       = x[4]  # lateral deviation
    dpsi    = x[5]  # heading deviation (btwn body x and path tangent)
    
    
    # Unpack controls
    delta   = u[0]  # steering angle
    kappa_f = u[1]  # front axle slip ratio
    kappa_r = u[2]  # rear axle slip ratio
    
    
    # Calculate normal loads
    fz_f = params["m"]*params["b"]*G/params["l"]
    fz_r = params["m"]*params["a"]*G/params["l"]
    
    
    # Calculate tire forces
    alpha_f = np.arctan2(uy + params["a"]*r, ux) - delta
    alpha_r = np.arctan2(uy - params["b"]*r, ux)
    
    fx_f, fy_f = compute_coupled_slip_forces(kappa_f, alpha_f, fz_f, params["Cx_f"], params["Cy_f"], params["mu_f"])
    fx_r, fy_r = compute_coupled_slip_forces(kappa_r, alpha_r, fz_r, params["Cx_r"], params["Cy_r"], params["mu_r"])
    
    
    # Evaluate equations of motion
    ux_dot = (-fy_f*np.sin(delta) + fx_f*np.cos(delta) + fx_r)/params["m"] + r*uy
    uy_dot = (fy_f*np.cos(delta) + fx_f*np.sin(delta) + fy_r)/params["m"] - r*ux
    r_dot  = (params["a"]*(fy_f*np.cos(delta) + fx_f*np.sin(delta)) - params["b"]*fy_r)/params["Izz"]

    s_dot = (ux*np.cos(dpsi) - uy*np.sin(dpsi))/(1-k_psi*e)
    e_dot = ux*np.sin(dpsi) + uy*np.cos(dpsi)
    dpsi_dot = r - k_psi*s_dot
    
    return np.array([ux_dot, uy_dot, r_dot, s_dot, e_dot, dpsi_dot])

In [ ]:
# Straight driving, then step the steering angle by +10 degrees

N = 1000 # number of discretization points
t_final = 10 # seconds
t_vals = np.linspace(0, t_final, N)
dt = t_vals[1] - t_vals[0]

x_vals = np.zeros((6, N))
u_vals = np.zeros((3, N))

# Initial conditions
x_vals[:, 0] = [10, 0, 0, 0, 0, 0]

# Step-steering
u_vals[0, :] = np.deg2rad(10)
u_vals[1, :] = 0.1  # front axle slip ratio
u_vals[2, :] = 0.1  # rear axle slip ratio

# Simulate with simple fwd-Euler
for i in range(N-1):
    x_dot = compute_x_dot(x_vals[:, i], u_vals[:, i], 0, params)
    x_vals[:, i+1] = x_vals[:, i] + dt*x_dot
    

# Plot the trajectories
fig1, ax1 = plt.subplots(3, 2, constrained_layout = True)
ax1[0, 0].plot(t_vals, x_vals[0, :])
ax1[1, 0].plot(t_vals, x_vals[1, :])
ax1[2, 0].plot(t_vals, x_vals[2, :])
ax1[0, 1].plot(t_vals, x_vals[3, :])
ax1[1, 1].plot(t_vals, x_vals[4, :])
ax1[2, 1].plot(t_vals, x_vals[5, :])
fig1.suptitle("States vs time\nux, uy, r (left column)\ns, e, dpsi (right column)")
plt.show()

In [ ]:
# Plot the overhead
# n.b.: This plot only works because we are relative to a straight line
fig2, ax2 = plt.subplots(constrained_layout = True)
ax2.plot(-x_vals[4, :], x_vals[3, :])
ax2.scatter(-x_vals[4, 0], x_vals[3, 0], marker = "x")
ax2.set_aspect("equal")
fig2.suptitle("Vehicle position (North vs East)")
plt.show()